In [10]:
import pandas as pd

# Load dataset
df = pd.read_csv('combined_wine_quality.csv')

# Filter high-scoring red and white wines
df_high_score = df[df['quality'] > 7]

alcohol_quality_stats = df['quality'].describe()
alcohol_quality_stats

count    6497.000000
mean        5.818378
std         0.873255
min         3.000000
25%         5.000000
50%         6.000000
75%         6.000000
max         9.000000
Name: quality, dtype: float64

In [6]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Initialize Dash app
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("High-Scoring Red vs White Wines", style={'text-align': 'center', 'color': '#4CAF50'}),

    html.Div([
        html.Label("Select X-axis Feature:", style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='x-axis-feature',
            options=[{'label': col, 'value': col} for col in df_high_score.columns if df_high_score[col].dtype != 'object'],
            value='alcohol',
            style={'width': '50%', 'margin-bottom': '10px'}
        ),

        html.Label("Select Y-axis Feature:", style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='y-axis-feature',
            options=[{'label': col, 'value': col} for col in df_high_score.columns if df_high_score[col].dtype != 'object'],
            value='pH',
            style={'width': '50%'}
        ),
    ], style={'margin': '20px'}),

    dcc.Graph(id='wine-graph', style={'margin-top': '20px'}),
])

# Callback to update the graph based on dropdown selection
@app.callback(
    Output('wine-graph', 'figure'),
    [Input('x-axis-feature', 'value'),
     Input('y-axis-feature', 'value')]
)
def update_graph(x_feature, y_feature):
    fig = px.scatter(
        df_high_score,
        x=x_feature,
        y=y_feature,
        color='type',
        symbol='type',
        size='quality',
        hover_data=['quality', 'volatile acidity', 'sulphates'],
        title=f"Comparison of High-Scoring Wines ({x_feature} vs {y_feature})"
    )
    fig.update_layout(
        title_font_size=20,
        xaxis_title=x_feature,
        yaxis_title=y_feature,
        legend_title="Wine Type",
        template="plotly_white"
    )
    return fig

# Run app
if __name__ == '__main__':
    app.run_server(debug=True)
